In [39]:
!pip install requests

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import urllib # package required to interact with live webpage
from datetime import date
import datetime
import time
import os

In [2]:
#To acquire country codes needed for the urls
file_url = 'https://spotifycharts.com/regional/global/daily/latest'

r = requests.get(file_url) 

with open("spotify_top200.html",'wb') as spotify_html: 
      spotify_html.write(r.content)

In [3]:
file_name = 'spotify_top200.html'
with open(file_name) as f:  #Windows users may need to add the option encoding='utf8'
    spotify_soup = BeautifulSoup(f, 'html.parser')

In [4]:
list_tag_list = spotify_soup.find_all('li')[0:66] #HTML with country names and country codes

In [5]:
#Create Country - country code dictionary
country_code_dict = {}
for t in list_tag_list:
    country_code_dict[t.text] = t['data-value']

In [7]:
countries = list(country_code_dict.keys())
print(len(countries))
print(countries[1])

66
United States


In [16]:
for country in [countries[1]]:
    print(country_code_dict[country])

us


In [20]:
end_date = datetime.datetime(2020, 12, 31) # end_date =
missing_dates = {}

# for country in countries: #input the indices here
for country in [countries[1]]:
    
    start_date = datetime.datetime(2019, 7, 1) # Start date = 
    
    country_code = country_code_dict[country]
    
    print('country', country)    
    print('country_code', country_code)
    
    while start_date < end_date:
        
        csv_url = 'https://spotifycharts.com/regional/' + country_code + '/daily/' + start_date.strftime("%Y-%m-%d") + '/download'
        req = requests.get(csv_url)

        url_content = req.content # download the csv file
        
        directory = 'spotify_files/' # Enter directory path
        path =  directory + country + '_' + start_date.strftime("%Y-%m-%d") + '.csv'

        csv_file = open(path, 'wb')
        print('---------Downloading--------')
        print(csv_url)
        csv_file.write(url_content)
        csv_file.close()
        
        try:
            df = pd.read_csv(path, header=1)
        except:
            print('csv file not loaded')
            print('Country:', country)
            print('Date:', start_date)
            
            if country in missing_dates:
                missing_dates[country].append(start_date)
            else:
                missing_dates[country] = [start_date]

            
            start_date += datetime.timedelta(days=1)
            time.sleep(2)
            continue
                  
        df['Country'] = country
        df['Date'] = start_date
        df = df[['Date', 'URL', 'Position','Track Name','Artist', 'Streams', 'Country']]
        df = df.rename(columns={'URL': 'Track URL'})
                
        df.to_csv(path, index=False)
        
        start_date += datetime.timedelta(days=1) #increment by one day

        time.sleep(2)
        
#         break
#     break
    


country United States
country_code us
---------Downloading--------
https://spotifycharts.com/regional/us/daily/2020-12-08/download
---------Downloading--------
https://spotifycharts.com/regional/us/daily/2020-12-09/download
---------Downloading--------
https://spotifycharts.com/regional/us/daily/2020-12-10/download
---------Downloading--------
https://spotifycharts.com/regional/us/daily/2020-12-11/download
---------Downloading--------
https://spotifycharts.com/regional/us/daily/2020-12-12/download
---------Downloading--------
https://spotifycharts.com/regional/us/daily/2020-12-13/download
---------Downloading--------
https://spotifycharts.com/regional/us/daily/2020-12-14/download
---------Downloading--------
https://spotifycharts.com/regional/us/daily/2020-12-15/download
---------Downloading--------
https://spotifycharts.com/regional/us/daily/2020-12-16/download
---------Downloading--------
https://spotifycharts.com/regional/us/daily/2020-12-17/download
---------Downloading--------
htt

In [53]:
import pandas as pd
import glob

path = r'./spotify_files' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)
    
frame = pd.concat(li, axis=0, ignore_index=True)
frame = frame[['Date', 'Track URL', 'Position', 'Track Name',
              'Artist', 'Streams', 'Country']]
frame
# frame[frame['Date'] == '2020-12-31']

,Date,Track URL,Position,Track Name,Artist,Streams,Country
0,2019-07-08,https://open.spotify.com/track/6vBdBCoOhKHiYDD...,1.0,Goodbyes (Feat. Young Thug),Post Malone,2083200.0,United States
1,2019-07-08,https://open.spotify.com/track/0TK2YIli7K1leLo...,2.0,Señorita,Shawn Mendes,1737128.0,United States
2,2019-07-08,https://open.spotify.com/track/6fTt0CH2t0mdeB2...,3.0,Panini,Lil Nas X,1423008.0,United States
3,2019-07-08,https://open.spotify.com/track/5ry2OE6R2zPQFDO...,4.0,Money In The Grave (Drake ft. Rick Ross),Drake,1381364.0,United States
4,2019-07-08,https://open.spotify.com/track/5qmq61DAAOUaW8A...,5.0,Truth Hurts,Lizzo,1248424.0,United States
...,...,...,...,...,...,...,...
10796,2019-07-05,https://open.spotify.com/track/1mqlc0vEP9mU1kZ...,196.0,September,"Earth, Wind & Fire",213999.0,United States
10797,2019-07-05,https://open.spotify.com/track/5KawlOMHjWeUjQt...,197.0,Boy With Luv (feat. Halsey),BTS,212995.0,United States
10798,2019-07-05,https://open.spotify.com/track/2374M0fQpWi3dLn...,198.0,Africa,TOTO,212517.0,United States
10799,2019-07-05,https://open.spotify.com/track/7sO5G9EABYOXQKN...,199.0,Ric Flair Drip (& Metro Boomin),Offset,212313.0,United States


In [ ]:
# frame.to_csv('2021.01.09 US Top 200 Missing Dates.csv')

In [25]:
data_top200 = pd.read_csv('../input/spotify_top200.csv')\
.drop_duplicates()
# data_top200['Date'] = pd.to_datetime(data_top200['Date'])

In [37]:
data_us = data_top200[data_top200['Country']=='United States']\
.drop(columns = ['Unnamed: 0'])\
.sort_values(by = ['Date', 'Position'])\
.reset_index()\
.drop(columns = ['index'])

# data_us['Track URI'] = data_us['Track URL'].str.replace(r'https://open.spotify.com/track/', '')

In [38]:
data_us

,Date,Track URL,Position,Track Name,Artist,Streams,Country
0,1/1/2017,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,1,Bad and Boujee (feat. Lil Uzi Vert),Migos,1371493,United States
1,1/1/2017,https://open.spotify.com/track/343YBumqHu19cGo...,2,Fake Love,Drake,1180074,United States
2,1/1/2017,https://open.spotify.com/track/5aAx2yezTd8zXrk...,3,Starboy,The Weeknd,1064351,United States
3,1/1/2017,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,4,Closer,The Chainsmokers,1010492,United States
4,1/1/2017,https://open.spotify.com/track/6fujklziTHa8uoM...,5,Black Beatles,Rae Sremmurd,874289,United States
...,...,...,...,...,...,...,...
280595,9/9/2018,https://open.spotify.com/track/4wFjTWCunQFKtuk...,196,MotorSport,Migos,190324,United States
280596,9/9/2018,https://open.spotify.com/track/6vsV4D8BM6PioRr...,197,you should see me in a crown,Billie Eilish,190101,United States
280597,9/9/2018,https://open.spotify.com/track/2eAZfqOm4EnOF9V...,198,The Way Life Goes (feat. Oh Wonder),Lil Uzi Vert,189630,United States
280598,9/9/2018,https://open.spotify.com/track/06EL94D0TA27Ik0...,199,The Way,Ariana Grande,188673,United States


In [54]:
complete = pd.concat([data_us, frame], axis=0, ignore_index=True)
complete

,Date,Track URL,Position,Track Name,Artist,Streams,Country
0,1/1/2017,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,1.0,Bad and Boujee (feat. Lil Uzi Vert),Migos,1371493.0,United States
1,1/1/2017,https://open.spotify.com/track/343YBumqHu19cGo...,2.0,Fake Love,Drake,1180074.0,United States
2,1/1/2017,https://open.spotify.com/track/5aAx2yezTd8zXrk...,3.0,Starboy,The Weeknd,1064351.0,United States
3,1/1/2017,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,4.0,Closer,The Chainsmokers,1010492.0,United States
4,1/1/2017,https://open.spotify.com/track/6fujklziTHa8uoM...,5.0,Black Beatles,Rae Sremmurd,874289.0,United States
...,...,...,...,...,...,...,...
291396,2019-07-05,https://open.spotify.com/track/1mqlc0vEP9mU1kZ...,196.0,September,"Earth, Wind & Fire",213999.0,United States
291397,2019-07-05,https://open.spotify.com/track/5KawlOMHjWeUjQt...,197.0,Boy With Luv (feat. Halsey),BTS,212995.0,United States
291398,2019-07-05,https://open.spotify.com/track/2374M0fQpWi3dLn...,198.0,Africa,TOTO,212517.0,United States
291399,2019-07-05,https://open.spotify.com/track/7sO5G9EABYOXQKN...,199.0,Ric Flair Drip (& Metro Boomin),Offset,212313.0,United States


In [55]:
complete2 = complete[['Date']].drop_duplicates()
complete2['Date'] = pd.to_datetime(complete2['Date'])
complete2['Year'] = complete2['Date'].dt.strftime('%Y')
complete2['Month'] = complete2['Date'].dt.strftime('%m')

In [56]:
complete2.groupby(['Year', 'Month']).count()

Date
Year Month      
2017 01       31
     02       28
     03       31
     04       30
     05       29
     06       29
     07       31
     08       31
     09       30
     10       31
     11       30
     12       31
2018 01       31
     02       28
     03       31
     04       30
     05       31
     06       30
     07       31
     08       31
     09       30
     10       31
     11       30
     12       31
2019 01       31
     02       28
     03       31
     04       30
     05       31
     06       30
     07       30
     08       31
     09       30
     10       31
     11       30
     12       31
2020 01       31
     02       29
     03       30
     04       30
     05       31
     06       30
     07       31
     08       31
     09       30
     10       31
     11       30
     12       31

In [57]:
complete2[complete2['Date'] >= '2020-12-01'].sort_values(by = 'Date')

,Date,Year,Month
169600,2020-12-01,2020,12
169800,2020-12-02,2020,12
170000,2020-12-03,2020,12
170200,2020-12-04,2020,12
170400,2020-12-05,2020,12
170600,2020-12-06,2020,12
170800,2020-12-07,2020,12
286201,2020-12-08,2020,12
286001,2020-12-09,2020,12
284801,2020-12-10,2020,12


In [59]:
complete.to_csv('../input/spotify_top200_us.csv')